# 缩减单元格

In [15]:
import pandas as pd
from math import radians, cos, sin, asin, sqrt
import numpy as np

In [16]:
# 经纬度计算距离
def geoDistance(lat1, lon1, lat2, lon2):
    # 将十进制度数转化为弧度
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    # haversine公式
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * asin(sqrt(a))
    r = 6371  # 地球平均半径，单位为公里
    return c * r


In [68]:
# 时间戳测试
t = 1452760800
print(t)
print(str(pd.to_datetime(t, unit='s')).split(' ')[0])

1452760800
2016-01-14


In [24]:
# 字典测试
airport = pd.read_csv('./dataset/airport.csv', encoding= 'gbk')
# 读取airport.csv文件为字典
airport_dict = {}
for i in range(len(airport)):
    airport_dict[airport['机场编码'][i]] = (airport['经度'][i], airport['纬度'][i])
print(airport_dict)

{'JDZ': (117.18366, 29.27324), 'PVG': (121.48941, 31.40527), 'TGO': (122.25615, 43.62244), 'YIW': (120.07468, 29.30558), 'XIY': (108.93425, 34.23053), 'XMN': (118.03394, 24.48405), 'URC': (88.31104, 43.36378), 'TYN': (112.48699, 37.94036), 'SZX': (113.88308, 22.55329), 'SYX': (109.7525, 18.40005), 'TAO': (120.39629, 36.30744), 'TNA': (116.75199, 36.55358), 'SJW': (114.53952, 38.03647), 'SHE': (123.46987, 41.80515), 'SHA': (121.48941, 31.40527), 'PEK': (116.23128, 40.22077), 'NKG': (118.8921, 31.32751), 'KWL': (110.30188, 25.31402), 'KMG': (102.82147, 24.88554), 'HRB': (126.95717, 45.54774), 'HET': (111.62299, 40.80772), 'HGH': (120.21201, 30.2084), 'FOC': (119.27345, 26.04769), 'CTU': (104.10194, 30.65984), 'CKG': (106.54041, 29.40268), 'HFE': (117.30794, 31.79322), 'CGO': (113.6401, 34.72468), 'TLQ': (89.18596, 42.94244), 'WUH': (114.02919, 30.58203)}


In [69]:
# 改为utf-8编码 读取dataset/flight.csv文件
origin_dataset = pd.read_csv('./dataset/flights.csv', encoding='gbk')
print('成功读取flights.csv文件')
# 删除ORIGIN_AIRPORT行中没有dataset/airports.csv中的IATA_CODE的行
airport = pd.read_csv('./dataset/airport.csv', encoding= 'gbk')
print('成功读取airport.csv文件')
select_airport = airport['机场编码'].values

# 将origin_dataset中的ORIGIN_AIRPORT列中的值与select_airport中的值进行比较，如果不在select_airport中，则删除该行
origin_dataset = origin_dataset[origin_dataset['出发机场'].isin(select_airport)]
print('成功删除ORIGIN_AIRPORT不在airports.csv中的行')
# 保存缩减后的数据集
origin_dataset = origin_dataset[origin_dataset['到达机场'].isin(select_airport)]


origin_dataset['计划出发日期'] = str(0)
origin_dataset['计划到达日期'] = str(0)
origin_dataset['实际出发日期'] = str(0)
origin_dataset['实际到达日期'] = str(0)
origin_dataset['出发延迟'] = 0
origin_dataset['到达延迟'] = 0
origin_dataset['出发经度'] = 0
origin_dataset['出发纬度'] = 0
origin_dataset['到达经度'] = 0
origin_dataset['到达纬度'] = 0
origin_dataset['距离'] = 0
# 时间码转换为时间格式
for i in range(len(origin_dataset)):
    origin_dataset['计划出发日期'].values[i] = str(pd.to_datetime(origin_dataset['计划出发'].values[i],unit='s')).split(' ')[0]
    origin_dataset['计划到达日期'].values[i] = str(pd.to_datetime(origin_dataset['计划到达'].values[i],unit='s')).split(' ')[0]
    origin_dataset['实际出发日期'].values[i] = str(pd.to_datetime(origin_dataset['实际出发'].values[i],unit='s')).split(' ')[0]
    origin_dataset['实际到达日期'].values[i] = str(pd.to_datetime(origin_dataset['实际到达'].values[i],unit='s')).split(' ')[0]
    if i%10000 == 0:
        print(i/len(origin_dataset)*100, '%')
'''
# 创建新的列
# 起飞延误时间和到达延误时间

    origin_dataset['出发延迟'] = (pd.to_datetime(origin_dataset['实际出发']) -  pd.to_datetime(origin_dataset['计划出发'])) / np.timedelta64(1, 'm')
    origin_dataset['到达延迟'] = (pd.to_datetime(origin_dataset['实际到达']) -  pd.to_datetime(origin_dataset['计划到达'])) / np.timedelta64(1, 'm')
    

# 通过airport_dict创建出发经纬度与到达经纬度
# 创建新的列

    origin_dataset['出发经度'].values[i] = airport_dict[origin_dataset['出发机场'].values[i]][0]
    origin_dataset['出发纬度'].values[i] = airport_dict[origin_dataset['出发机场'].values[i]][1]
    origin_dataset['到达经度'].values[i] = airport_dict[origin_dataset['到达机场'].values[i]][0]
    origin_dataset['到达纬度'].values[i] = airport_dict[origin_dataset['到达机场'].values[i]][1]

# 通过经纬度计算距离

    origin_dataset['距离'].values[i] = geoDistance(origin_dataset['出发经度'].values[i], origin_dataset['出发纬度'].values[i], origin_dataset['到达经度'].values[i], origin_dataset['到达纬度'].values[i])
    
# origin_dataset['距离'] = geoDistance(airport[airport['机场编码'].values == origin_dataset['出发机场'].values]['经度'], airport[airport['机场编码'].values == origin_dataset['出发机场'].values]['纬度'], airport[airport['机场编码'].values == origin_dataset['到达机场'].values]['经度'], airport[airport['机场编码'].values == origin_dataset['到达机场'].values]['纬度'])
'''

origin_dataset.to_csv('./dataset/flight_reduced.csv', index=False)

成功读取flights.csv文件
成功读取airport.csv文件
成功删除ORIGIN_AIRPORT不在airports.csv中的行
0.0 %
0.36373238898705623 %
0.7274647779741125 %
1.0911971669611686 %
1.454929555948225 %
1.818661944935281 %
2.1823943339223373 %
2.5461267229093933 %
2.90985911189645 %
3.2735915008835064 %
3.637323889870562 %
4.001056278857618 %
4.364788667844675 %
4.728521056831731 %
5.092253445818787 %
5.455985834805843 %
5.8197182237929 %
6.183450612779956 %
6.547183001767013 %
6.910915390754068 %
7.274647779741124 %
7.6383801687281805 %
8.002112557715236 %
8.365844946702293 %
8.72957733568935 %
9.093309724676406 %
9.457042113663462 %
9.820774502650519 %
10.184506891637573 %
10.54823928062463 %
10.911971669611686 %
11.275704058598743 %
11.6394364475858 %
12.003168836572856 %
12.366901225559912 %
12.730633614546969 %
13.094366003534025 %
13.458098392521082 %
13.821830781508137 %
14.185563170495191 %
14.549295559482248 %
14.913027948469304 %
15.276760337456361 %
15.640492726443417 %
16.004225115430472 %
16.36795750441753 %
16.7

In [71]:
# 读取./dataset/flight_reduced.csv
reduced_dataset = pd.read_csv('./dataset/flight_reduced.csv')
# 删除同一出发机场重复的计划出发的列
reduced_dataset = reduced_dataset.drop_duplicates(subset=['出发机场', '计划出发日期'], keep='first')
# 保存去重后的数据集
reduced_dataset.to_csv('./dataset/flight_reduced_unique.csv', index=False)

In [78]:
# 读取./dataset/flight_reduced_unique.csv
reduced_unique_dataset = pd.read_csv('./dataset/flight_reduced_unique.csv')
# 删除取消的航班
reduced_unique_dataset = reduced_unique_dataset[reduced_unique_dataset['是否取消'] != '取消']
# 保存去重后的数据集
reduced_unique_dataset.to_csv('./dataset/flight_reduced_unique.csv', index=False)

In [80]:
# 读取./dataset/flight_reduced_unique.csv
reduced_unique_dataset = pd.read_csv('./dataset/flight_reduced_unique.csv')
# 计算延迟时间
reduced_unique_dataset['出发延迟'] = (pd.to_datetime(reduced_unique_dataset['实际出发'], unit='s') -  pd.to_datetime(reduced_unique_dataset['计划出发'], unit='s')) / np.timedelta64(1, 'm')
reduced_unique_dataset['到达延迟'] = (pd.to_datetime(reduced_unique_dataset['实际到达'], unit='s') -  pd.to_datetime(reduced_unique_dataset['计划到达'], unit='s')) / np.timedelta64(1, 'm')
# 计算出发经纬度与到达经纬度
for i in range(len(reduced_unique_dataset)):
    # 通过airport_dict创建出发经纬度与到达经纬度
    
    reduced_unique_dataset['出发经度'].values[i] = airport_dict[reduced_unique_dataset['出发机场'].values[i]][0]
    reduced_unique_dataset['出发纬度'].values[i] = airport_dict[reduced_unique_dataset['出发机场'].values[i]][1]
    reduced_unique_dataset['到达经度'].values[i] = airport_dict[reduced_unique_dataset['到达机场'].values[i]][0]
    reduced_unique_dataset['到达纬度'].values[i] = airport_dict[reduced_unique_dataset['到达机场'].values[i]][1]
    # 通过经纬度计算距离
    
    reduced_unique_dataset['距离'].values[i] = geoDistance(reduced_unique_dataset['出发经度'].values[i], reduced_unique_dataset['出发纬度'].values[i], reduced_unique_dataset['到达经度'].values[i], reduced_unique_dataset['到达纬度'].values[i])
# 保存计算后的数据集
reduced_unique_dataset.to_csv('./dataset/flight_reduced_unique_calculated.csv', index=False)